In [ ]:
import warnings
warnings.filterwarnings("ignore")

#### libraries

In [104]:
import pandas as pd
import numpy as np
import copy

#### inputs

In [130]:
min_sup=float(input("Enter minimum support:  "))
min_conf=float(input("Enter minimum confidence:  "))

Enter minimum support:  0.01
Enter minimum confidence:  0.2


#### functions

In [119]:
def load_dataset():
    data_frame = pd.DataFrame()
    data_frame=pd.read_csv('market_data.csv', header=None )
    number_of_all_transactions=len(data_frame)
    return data_frame,number_of_all_transactions

In [120]:
def delete_zero_occurance_items(data_frame):
    data_frame = data_frame[data_frame.occurance != 0]
    data_frame.index = range(len(data_frame))
    return data_frame

In [121]:
def filter_by_support(dff):
    df_filtered = dff[dff["support"]>= float(min_sup)]
    df_filtered.index = range(len(df_filtered))
    return df_filtered

In [109]:
def filter_by_confidence(rule):
    rule = rule[rule["confidence"] >= float(min_conf)]
    rule.index = range(len(rule))
    return rule

In [122]:
def count_occurance_of_items(data_frame):
    number_of_all_transactions=len(data_frame)
    all_items = pd.unique(data_frame.values.ravel())
    df = pd.DataFrame()
    df["item"]=" "
    df["occurance"]=" "
    df["support"] = " "
    df["lenght"]=" "   
    index=0
    for item in all_items:
        occurenece_in_each_row=data_frame[data_frame == item].count()
        df.at[index,"occurance"]=occurenece_in_each_row.sum()
        df.at[index,"item"]=[item]
        df.at[index,"support"]=df.at[index,"occurance"]/number_of_all_transactions
        df.at[index,"lenght"]=len([item])
        index=index+1
    df=delete_zero_occurance_items(df)
    #print("⬇ table of itemset occurance ⬇ ") 
    #print(df)
    return df,list(df["item"])

In [123]:
def create_new_itemset(items,df):
    data_frame = pd.DataFrame()
    data_frame["item"]=""
    added=[]
    frequent_items=list(df["item"])
    different_items=copy.deepcopy(items)
    index=0
    new_items=[]
    i=0
    while (i<len(different_items)):
        j=0
        while (j<len(frequent_items)-1): 
            if not(different_items[i][0] in frequent_items[j]):
                new_frequent_items=(frequent_items[j]+different_items[i])
                if not(set(new_frequent_items) in added):
                    data_frame.at[index,"item"]=new_frequent_items
                    added.append(set(new_frequent_items))
                    index=index+1
            j=j+1    
        different_items.remove(different_items[i])
        i=i+1
    #print("⬇ table of itemset occurance ⬇" )    
    #display(data_frame)
    return data_frame

In [124]:
def calculate_occurance_and_support(ddf,data_frame):
    number_of_all_transactions=len(data_frame)
    index=0
    ddf["occurance"] = " "
    ddf["support"] = " "
    ddf["lenght"]=" "   
    for item in ddf["item"]:
        rows=[]
        for items in item:
            row, columns = np.where(data_frame == items) 
            rows.append(row)  
        intersection=len(set.intersection(*map(set,rows)))
        ddf.at[index,"occurance"]=intersection
        ddf.at[index,"support"]=intersection/number_of_all_transactions
        ddf.at[index,"lenght"]=len(item)
        index=index+1
    df_filtered=filter_by_support(ddf)
    #display(df_filtered)
    return df_filtered


In [125]:
def create_rules_with_one_if_condition(new_occurance_table,occurance_table,number_of_all_transaction):
    column_names = ["item", "support", "confidence","if condition","then condition"]
    rule = pd.DataFrame(columns = column_names)
    index=0
    i=0
    for items in new_occurance_table["item"]:
        for item in items:
            rule.at[index,"item"]=items
            rule.at[index,"support"]=new_occurance_table.at[i,"support"]
            then_list=copy.deepcopy(items)
            rule.at[index,"if condition"]=[item]
            for x in range(len(occurance_table["item"])):
                if occurance_table.at[x,"item"]==[item]:
                    rule.at[index,"confidence"]=number_of_all_transaction*(rule.at[index,"support"])/(occurance_table.at[x,"occurance"])  
            then_list.remove(item)
            rule.at[index,"then condition"]=then_list
            index=index+1
        i=i+1
    return rule,index

In [126]:
def remove_from_then_condition_add_to_if_condition(rule,index,new_occurance_table,occurance_table,number_of_all_transaction):
    previous_index=0
    for items in rule["then condition"]:
        for item in items:
            rule.at[index,"item"]=rule.at[previous_index,"item"]
            rule.at[index,"support"]=rule.at[previous_index,"support"]
            new_if=copy.deepcopy(rule.at[previous_index,"if condition"])
            if len(new_if)!=len(rule.at[previous_index,"item"])-1:
                then_list=copy.deepcopy(items)
                new_if.append(item)
                rule.at[index,"if condition"]=new_if
                then_list.remove(item)
                rule.at[index,"then condition"]=then_list
                for x in range(len(occurance_table["item"])):
                    if occurance_table.at[x,"item"]==[item]:
                        rule.at[index,"confidence"]=number_of_all_transaction*(rule.at[index,"support"])/(occurance_table.at[x,"occurance"])  
                previous_index=previous_index+1
                index=index+1
    return rule

In [127]:
def create_rule(new_occurance_table,occurance_table,number_of_all_transaction):
    rule,index=create_rules_with_one_if_condition(new_occurance_table,occurance_table,number_of_all_transaction)
    rule=remove_from_then_condition_add_to_if_condition(rule,index,new_occurance_table,occurance_table,number_of_all_transaction)
    rule = rule.dropna()
    df_filtered=filter_by_support(rule)
    df_filtered=filter_by_confidence(df_filtered)
    #print("⬇ table of rules ⬇ ")
    return df_filtered

In [128]:
all_subsets = pd.DataFrame()
all_rules = pd.DataFrame()

In [134]:
data_frame,number_of_all_transaction=load_dataset()
occurance_table,different_items=count_occurance_of_items(data_frame)
repeat=len(different_items)
all_subsets = pd.DataFrame()
all_rules = pd.DataFrame()
for _ in range(repeat):
    all_subsets = all_subsets.append(occurance_table, ignore_index=True)
    df=create_new_itemset(different_items,occurance_table)
    occurance_table=calculate_occurance_and_support(df,data_frame)
    if occurance_table.empty:
        break
    rules=create_rule(occurance_table,all_subsets,number_of_all_transaction)
    #print(rules)
    all_rules=all_rules.append(rules ,ignore_index=True)
print(all_rules)

                                     item   support confidence  \
0                 [mineral water, shrimp]  0.023597   0.330224   
1                          [milk, shrimp]  0.017598   0.246269   
2             [frozen vegetables, shrimp]  0.016664   0.233209   
3                     [spaghetti, shrimp]  0.021197   0.296642   
4                     [chocolate, shrimp]  0.017998   0.251866   
..                                    ...       ...        ...   
85             [chocolate, grated cheese]  0.010932   0.208651   
86           [ground beef, grated cheese]  0.011332   0.216285   
87               [french fries, escalope]  0.016398   0.206723   
88                  [chocolate, escalope]  0.017598   0.221849   
89  [chocolate, spaghetti, mineral water]  0.015865   0.240891   

                  if condition       then condition  
0                     [shrimp]      [mineral water]  
1                     [shrimp]               [milk]  
2                     [shrimp]  [frozen veget